<a href="https://colab.research.google.com/github/EgorSolovei/Decision-trees-in-nuclear-physics/blob/main/loop_create_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# чтобы не вылазило предупреждение, с которым я потом как-нибудь разберусь
pd.options.mode.chained_assignment = None

In [3]:
data_1_df = pd.read_csv('Data_1.csv', sep=';', index_col=False)

# убрали пока не нужные колонки
data_1_df = data_1_df.drop(columns=['baryon_number', 'impulse_z_lab', 'param_5'])

# заменим все NaN, предполагая, что такого значения в массе нет, так как она должна быть положительна
data_1_df = data_1_df.fillna(-1)

# проверка сколько у нас событий (то есть мест, где раньше стояли NaN)
data_1_df.query('mass == -1').agg({'mass': 'count'}) # посмотреть сколько в нашем файле событий

mass    2000
dtype: int64

In [4]:
data_1_df.shape

(1495919, 8)

In [5]:
# создание пустого DataFrame с нужными колонками
lst_col = ['class']

for i in range(1, 4):
    for j in range(1, 33):
        lst_col.append(f'sns_sct_{i}_{j}_plus')
        lst_col.append(f'sns_sct_{i}_{j}_minus')

data_res = pd.DataFrame(columns=lst_col)

In [6]:
# подвинем немного правую границу 4 класса, чтобы не возникло проблем на границе
borders = {1: [0, 7], 2: [7, 9.8], 3: [9.8, 12], 4: [12, 16.4]}


# основная функция

def get_vector_feature(data, idx_event):
    # функция определения класса
    def define_class_impact_param(dist):
        for i in range(1, 5):
            if borders[i][0] <= dist < borders[i][1]:
                return i

    # создадим функцию, которая будет брать только одно событие - данные с одного столкновения ядер.
    # эти данные и нужно будет преобразовать в вектор признаков

    def split_data_event(df, idx):
        data_temp = pd.DataFrame()
        for i in range(df.shape[0]):
            if df.mass[i] == -1 and df.particle_charge[i] == idx:
                cnt_particle = df.lepton_number[i]  # количество частиц в столкновении
                data_temp = df[i + 1: i + 1 + cnt_particle]  # вырезаем нужную часть

                class_param = define_class_impact_param(df.strangeness[i])  # класс прицельного параметра
                data_temp.insert(0, "class_param", class_param)
                return data_temp

    data_for_vec = split_data_event(data, idx_event)  # теперь эти данные нужно превратить в вектор

    # Фильтруем данные
    # -----------------------------------------------------------------

    # больше нам эти колонки не нужны, так как мы уже сделали классификацию по прицельному параметру
    data_for_vec = data_for_vec.drop(columns=['lepton_number', 'strangeness', 'type_of_particles'])
    data_for_vec = data_for_vec.query("particle_charge != 0")

    # Импульс
    # -----------------------------------------------------------------

    # посчитали суммарный импульс для каждой частицы
    # def total_impulse(df):
    #     total = np.power(df.impulse_x, 2) + np.power(df.impulse_y, 2) + np.power(df.impulse_z, 2)
    #     return np.sqrt(total)
    #
    # data_for_vec['modul_sum_impulse'] = total_impulse(data_for_vec)  # тут pandas ругается на что-то

    def total_impulse(row):
        total = np.power(row.impulse_x, 2) + np.power(row.impulse_y, 2) + np.power(row.impulse_z, 2)
        return np.sqrt(total)

    data_for_vec['modul_sum_impulse'] = data_for_vec.apply(total_impulse, axis=1)  # тут pandas ругается на что-то

    # Прилетит ли частица на какой-либо датчик
    # -----------------------------------------------------------------

    R = 2  # внешний радиус датчика
    r_cut = 0.3  # внутренний радиус датчика
    dist = [1, 3, 5]  # расстояние до соответствующего датчика

    alpha_0 = np.arctan(r_cut / dist[2])
    beta_0 = np.arctan(R / dist[0])

    def to_sensor(data):
        arccos = np.arccos(data.impulse_z / data.modul_sum_impulse)
        # положительное положение датчиков
        result_plus = arccos.between(alpha_0, beta_0, inclusive="neither")
        # отрицательное положение датчиков
        result_minus = arccos.between(np.pi - beta_0, np.pi - alpha_0, inclusive="neither")
        return result_plus | result_minus

    data_for_vec['to_any_sensor'] = to_sensor(data_for_vec)  # тут pandas ругается на что-то
    data_for_vec = data_for_vec.query("to_any_sensor == True")  # убираем те частицы, которые не долетаю ни до одного датчика

    # уберём колонки, которые уже не пригодятся
    # здесь каждая частица долетит до какого-нибудь датчика. Поэтому to_any_sensor уже не нужна
    data_for_vec = data_for_vec.drop(columns=["particle_charge", "to_any_sensor"])

    # На какой датчик прилетела частица
    # -----------------------------------------------------------------

    alpha_i = [np.arctan(r_cut / dist_i) for dist_i in dist]  # минимальные углы для i датчика
    beta_i = [np.arctan(R / dist_i) for dist_i in dist]  # максимальный угол для i датчика

    # создадим колонку с арккосинусом угла тета
    data_for_vec["arccos_theta"] = np.arccos(data_for_vec.impulse_z / data_for_vec.modul_sum_impulse)

    # создаём колонку для каждого датчика и делаем проверку по углам.
    for j in range(3):
        name_column = "sensor_" + str(j + 1)
        data_for_vec[name_column] = 0

        # этой функции передаём арккосинус угла и направление датчика
        def between_corner(arccos_series, direct):
            if direct == 1:
                return arccos_series.between(alpha_i[j], beta_i[j])
            elif direct == -1:
                return arccos_series.between(np.pi - beta_i[j], np.pi - alpha_i[j])

        data_for_vec.loc[between_corner(data_for_vec.arccos_theta, 1), name_column] = 1
        data_for_vec.loc[between_corner(data_for_vec.arccos_theta, -1), name_column] = -1

    # Разбиение на сектора
    # -----------------------------------------------------------------

    all_r = [r_cut, 0.7, 1.2, 1.6, R]  # разбиение на кольца датчика
    dist = [1, 3, 5]  # расстояние до соответствующего датчика

    """
    Определяем номер кольца попадания частицы для каждого датчика
    Внешний цикл создаёт колонку для каждой пары датчика, в которой лежат нули.
    Внутренний цикл проверяет в какое кольцо на датчике прилетает частица [1, 2, 3, 4] - 4 кольца
    Функция во внутреннем цикле работает с отбором по углам
    Работает так, как и задумывалось, потому что если частица попала на датчик, то есть номер кольца 
    попадания, в случае, если частица не попала на датчик, то как и ожидалось, там 0 
    """
    for i in range(3):
        name_column = "number_of_ring_" + str(i + 1)
        data_for_vec[name_column] = 0

        for j in range(4):
            alpha_ring = [np.arctan(r / dist[i]) for r in all_r]

            def between_corner(arccos_series, direct):  # нужно подумать над углами в between
                if direct == 1:
                    return arccos_series.between(alpha_ring[j], alpha_ring[j + 1])
                elif direct == -1:
                    return arccos_series.between(np.pi - alpha_ring[j + 1], np.pi - alpha_ring[j])

            data_for_vec.loc[between_corner(data_for_vec.arccos_theta, 1), name_column] = j + 1
            data_for_vec.loc[between_corner(data_for_vec.arccos_theta, -1), name_column] = j + 1

    # цикл подобен коду выше, только теперь это делается для части окружности
    data_for_vec["arctg_phi"] = np.arctan(data_for_vec.impulse_y / data_for_vec.impulse_x)

    for i in range(3):
        name_column = "number_of_piece_" + str(i + 1)
        column_sensor = "sensor_" + str(i + 1)
        data_for_vec[name_column] = 0

        def between_angle(left_border, rigth_border):
            result = data_for_vec.arctg_phi.between(left_border, rigth_border)
            return result

        """
        Ща будет колхоз. Надо бы как-то переписать это
        Условия по которым идёт отбор: 
        1) Частица должна прилететь на какой-то датчик: data_for_vec[column_sensor] != 0
        2) Отбор по знаку impulse_x, так как это определяет нужную половину датчика (правую или левую)
        3) И проверка на какой сектор прилетает частица: between_angle
        """
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x > 0) & (between_angle(-np.pi / 2, -np.pi / 4)), name_column] = 7
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x > 0) & (between_angle(-np.pi / 4, 0)), name_column] = 8
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x > 0) & (between_angle(0, np.pi / 4)), name_column] = 1
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x > 0) & (between_angle(np.pi / 4, np.pi / 2)), name_column] = 2

        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x < 0) & (between_angle(-np.pi / 2, -np.pi / 4)), name_column] = 3
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x < 0) & (between_angle(-np.pi / 4, 0)), name_column] = 4
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x < 0) & (between_angle(0, np.pi / 4)), name_column] = 5
        data_for_vec.loc[(data_for_vec[column_sensor] != 0) & (data_for_vec.impulse_x < 0) & (between_angle(np.pi / 4, np.pi / 2)), name_column] = 6

    # алгоритм определения сектора по кольцу попадания и куску по фи
    data_for_vec["sector_1"] = (data_for_vec.number_of_ring_1 - 1) + data_for_vec.number_of_piece_1
    data_for_vec["sector_2"] = (data_for_vec.number_of_ring_2 - 1) + data_for_vec.number_of_piece_2
    data_for_vec["sector_3"] = (data_for_vec.number_of_ring_3 - 1) + data_for_vec.number_of_piece_3

    # алгоритм работает неправильно в случае number_of_ring_i == 0, поэтому так...
    data_for_vec.sector_1 = data_for_vec.sector_1.replace(-1, 0)
    data_for_vec.sector_2 = data_for_vec.sector_2.replace(-1, 0)
    data_for_vec.sector_3 = data_for_vec.sector_3.replace(-1, 0)

    # уберём все вспомогательные колонки
    data_for_vec = data_for_vec.drop(columns=['number_of_ring_1', 'number_of_ring_2', 'number_of_ring_2',
                                              'number_of_piece_1', 'number_of_piece_2', 'number_of_piece_3',
                                              'arctg_phi', 'arccos_theta'])

    # переставим колонки в приятном порядке
    data_for_vec = data_for_vec[['class_param', 'impulse_x', 'impulse_y', 'impulse_z',
                                 'modul_sum_impulse', 'mass', 'sensor_1', 'sector_1',
                                 'sensor_2', 'sector_2', 'sensor_3', 'sector_3']]

    # Разбиение на сектора
    # -----------------------------------------------------------------
    data_for_vec["velocity"] = data_for_vec.impulse_z / np.sqrt(data_for_vec.mass ** 2 + data_for_vec.modul_sum_impulse ** 2)

    """
    нужно обрабатывать случаи, когда частица не прилетает на датчик
    умножим расстояние на индикатор прилёта частицы - 1, -1 или 0.
    в случае, если частица НЕ прилетела на датчик, то и время пролёта будет 0
    возьмём модуль, так как для времен не имеет значение, в каком направлении летит частица
    """
    data_for_vec["time_1"] = abs((dist[0] * data_for_vec.sensor_1) / data_for_vec.velocity)
    data_for_vec["time_2"] = abs((dist[1] * data_for_vec.sensor_2) / data_for_vec.velocity)
    data_for_vec["time_3"] = abs((dist[2] * data_for_vec.sensor_3) / data_for_vec.velocity)

    # уберём лишнее и сделаем в приятном виде
    data_for_vec = data_for_vec.drop(columns=["velocity", "impulse_x", "impulse_y", "impulse_z",
                                              "modul_sum_impulse", "mass"])

    data_for_vec = data_for_vec[["sensor_1", "sector_1", "time_1",
                                 "sensor_2", "sector_2", "time_2",
                                 "sensor_3", "sector_3", "time_3", "class_param"]]

    # Создание результирующего вектора
    # -----------------------------------------------------------------

    res = pd.Series([0] * 193, index=lst_col)  # см. 13 - 18 строку

    # заполнение вектора res
    def position_in_res_vec(df):
        res['class'] = df.class_param.values[0]
        for i in range(1, 4):
            for j in range(1, 33):
                # создадим условие для нахождения минимального времени пролёта
                condition_plus = (df[f'sensor_{i}'] == 1) & (df[f'sector_{i}'] == j)
                condition_minus = (df[f'sensor_{i}'] == -1) & (df[f'sector_{i}'] == j)

                # сортируем по времени и берём самое меньшее время
                min_data_plus = df[condition_plus].sort_values(f'time_{i}')
                min_data_minus = df[condition_minus].sort_values(f'time_{i}')

                # возможны случаи, когда ни одна частица не пролетела через датчик
                if min_data_plus.shape[0] != 0:
                    res[f'sns_sct_{i}_{j}_plus'] = min_data_plus[f'time_{i}'].values[0]

                if min_data_minus.shape[0] != 0:
                    res[f'sns_sct_{i}_{j}_minus'] = min_data_minus[f'time_{i}'].values[0]

    # обработка случая, когда после фильтрации не осталось никаких частиц попавших на датчик. В этом случае вернём нулевой вектор
    if data_for_vec.shape[0] == 0:
        return res.values
    else:
        position_in_res_vec(data_for_vec)
        return res.values


In [7]:
for id in range(1, 2001):
    vec = get_vector_feature(data_1_df, id)
    data_res.loc[len(data_res.index)] = vec  # добавление вектора в конец
    if id % 100 == 0:
        print(f"Выполнено до события: {id}")
        data_res.to_csv("result_data_1.csv", index=False) # пусть файл перезаписывается

Выполнено до события: 100
Выполнено до события: 200
Выполнено до события: 300
Выполнено до события: 400
Выполнено до события: 500
Выполнено до события: 600
Выполнено до события: 700
Выполнено до события: 800
Выполнено до события: 900
Выполнено до события: 1000
Выполнено до события: 1100
Выполнено до события: 1200


KeyboardInterrupt: ignored

In [ ]:
data_res.head(-5)

,class,sns_sct_1_1_plus,sns_sct_1_1_minus,sns_sct_1_2_plus,sns_sct_1_2_minus,sns_sct_1_3_plus,sns_sct_1_3_minus,sns_sct_1_4_plus,sns_sct_1_4_minus,sns_sct_1_5_plus,...,sns_sct_3_28_plus,sns_sct_3_28_minus,sns_sct_3_29_plus,sns_sct_3_29_minus,sns_sct_3_30_plus,sns_sct_3_30_minus,sns_sct_3_31_plus,sns_sct_3_31_minus,sns_sct_3_32_plus,sns_sct_3_32_minus
0,2.0,1.070476,1.134191,1.061568,1.063757,1.056732,1.071109,1.063262,1.074886,1.249140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,1.062580,0.000000,0.000000,0.000000,1.610508,1.232604,0.000000,1.184101,1.320536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.067535,1.051729,1.054481,1.057698,1.055646,1.062432,1.071453,1.064985,1.065528,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.088980,1.055896,1.061196,1.138536,1.091437,1.074876,1.061321,1.082646,1.095676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,1.081317,1.061869,1.053991,1.112041,1.073001,1.089580,1.092859,1.097151,1.048954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,2.0,1.068088,1.064568,1.059707,1.054472,1.151175,1.053891,1.122632,1.078557,1.065256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1569,4.0,0.000000,0.000000,0.000000,1.100588,0.000000,0.000000,0.000000,1.662233,1.078660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1570,3.0,2.143813,1.285336,1.098017,1.152543,1.097588,1.076426,1.405858,1.054741,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1571,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_res.shape

(1578, 193)

In [ ]:
df = pd.read_csv('result_data_1.csv', sep=';')
df # ужасно

,"class,sns_sct_1_1_plus,sns_sct_1_1_minus,sns_sct_1_2_plus,sns_sct_1_2_minus,sns_sct_1_3_plus,sns_sct_1_3_minus,sns_sct_1_4_plus,sns_sct_1_4_minus,sns_sct_1_5_plus,sns_sct_1_5_minus,sns_sct_1_6_plus,sns_sct_1_6_minus,sns_sct_1_7_plus,sns_sct_1_7_minus,sns_sct_1_8_plus,sns_sct_1_8_minus,sns_sct_1_9_plus,sns_sct_1_9_minus,sns_sct_1_10_plus,sns_sct_1_10_minus,sns_sct_1_11_plus,sns_sct_1_11_minus,sns_sct_1_12_plus,sns_sct_1_12_minus,sns_sct_1_13_plus,sns_sct_1_13_minus,sns_sct_1_14_plus,sns_sct_1_14_minus,sns_sct_1_15_plus,sns_sct_1_15_minus,sns_sct_1_16_plus,sns_sct_1_16_minus,sns_sct_1_17_plus,sns_sct_1_17_minus,sns_sct_1_18_plus,sns_sct_1_18_minus,sns_sct_1_19_plus,sns_sct_1_19_minus,sns_sct_1_20_plus,sns_sct_1_20_minus,sns_sct_1_21_plus,sns_sct_1_21_minus,sns_sct_1_22_plus,sns_sct_1_22_minus,sns_sct_1_23_plus,sns_sct_1_23_minus,sns_sct_1_24_plus,sns_sct_1_24_minus,sns_sct_1_25_plus,sns_sct_1_25_minus,sns_sct_1_26_plus,sns_sct_1_26_minus,sns_sct_1_27_plus,sns_sct_1_27_minus,sns_sct_1_28_plus,sns_sct_1_28_minus,sns_sct_1_29_plus,sns_sct_1_29_minus,sns_sct_1_30_plus,sns_sct_1_30_minus,sns_sct_1_31_plus,sns_sct_1_31_minus,sns_sct_1_32_plus,sns_sct_1_32_minus,sns_sct_2_1_plus,sns_sct_2_1_minus,sns_sct_2_2_plus,sns_sct_2_2_minus,sns_sct_2_3_plus,sns_sct_2_3_minus,sns_sct_2_4_plus,sns_sct_2_4_minus,sns_sct_2_5_plus,sns_sct_2_5_minus,sns_sct_2_6_plus,sns_sct_2_6_minus,sns_sct_2_7_plus,sns_sct_2_7_minus,sns_sct_2_8_plus,sns_sct_2_8_minus,sns_sct_2_9_plus,sns_sct_2_9_minus,sns_sct_2_10_plus,sns_sct_2_10_minus,sns_sct_2_11_plus,sns_sct_2_11_minus,sns_sct_2_12_plus,sns_sct_2_12_minus,sns_sct_2_13_plus,sns_sct_2_13_minus,sns_sct_2_14_plus,sns_sct_2_14_minus,sns_sct_2_15_plus,sns_sct_2_15_minus,sns_sct_2_16_plus,sns_sct_2_16_minus,sns_sct_2_17_plus,sns_sct_2_17_minus,sns_sct_2_18_plus,sns_sct_2_18_minus,sns_sct_2_19_plus,sns_sct_2_19_minus,sns_sct_2_20_plus,sns_sct_2_20_minus,sns_sct_2_21_plus,sns_sct_2_21_minus,sns_sct_2_22_plus,sns_sct_2_22_minus,sns_sct_2_23_plus,sns_sct_2_23_minus,sns_sct_2_24_plus,sns_sct_2_24_minus,sns_sct_2_25_plus,sns_sct_2_25_minus,sns_sct_2_26_plus,sns_sct_2_26_minus,sns_sct_2_27_plus,sns_sct_2_27_minus,sns_sct_2_28_plus,sns_sct_2_28_minus,sns_sct_2_29_plus,sns_sct_2_29_minus,sns_sct_2_30_plus,sns_sct_2_30_minus,sns_sct_2_31_plus,sns_sct_2_31_minus,sns_sct_2_32_plus,sns_sct_2_32_minus,sns_sct_3_1_plus,sns_sct_3_1_minus,sns_sct_3_2_plus,sns_sct_3_2_minus,sns_sct_3_3_plus,sns_sct_3_3_minus,sns_sct_3_4_plus,sns_sct_3_4_minus,sns_sct_3_5_plus,sns_sct_3_5_minus,sns_sct_3_6_plus,sns_sct_3_6_minus,sns_sct_3_7_plus,sns_sct_3_7_minus,sns_sct_3_8_plus,sns_sct_3_8_minus,sns_sct_3_9_plus,sns_sct_3_9_minus,sns_sct_3_10_plus,sns_sct_3_10_minus,sns_sct_3_11_plus,sns_sct_3_11_minus,sns_sct_3_12_plus,sns_sct_3_12_minus,sns_sct_3_13_plus,sns_sct_3_13_minus,sns_sct_3_14_plus,sns_sct_3_14_minus,sns_sct_3_15_plus,sns_sct_3_15_minus,sns_sct_3_16_plus,sns_sct_3_16_minus,sns_sct_3_17_plus,sns_sct_3_17_minus,sns_sct_3_18_plus,sns_sct_3_18_minus,sns_sct_3_19_plus,sns_sct_3_19_minus,sns_sct_3_20_plus,sns_sct_3_20_minus,sns_sct_3_21_plus,sns_sct_3_21_minus,sns_sct_3_22_plus,sns_sct_3_22_minus,sns_sct_3_23_plus,sns_sct_3_23_minus,sns_sct_3_24_plus,sns_sct_3_24_minus,sns_sct_3_25_plus,sns_sct_3_25_minus,sns_sct_3_26_plus,sns_sct_3_26_minus,sns_sct_3_27_plus,sns_sct_3_27_minus,sns_sct_3_28_plus,sns_sct_3_28_minus,sns_sct_3_29_plus,sns_sct_3_29_minus,sns_sct_3_30_plus,sns_sct_3_30_minus,sns_sct_3_31_plus,sns_sct_3_31_minus,sns_sct_3_32_plus,sns_sct_3_32_minus"
0,"2.0,1.0704757978662904,1.1341911725281943,1.06..."
1,"4.0,1.0625803900088375,0.0,0.0,0.0,1.610508381..."
2,"1.0,1.0675353421694578,1.0517294568970668,1.05..."
3,"1.0,1.0889797111591173,1.0558962875257725,1.06..."
4,"2.0,1.0813165978423662,1.0618692397385567,1.05..."
...,...
1495,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...."
1496,"1.0,1.082441671741347,1.0555889090389643,1.054..."
1497,"4.0,0.0,1.0831414683284895,2.5135208871945824,..."
1498,"1.0,1.0696696092981093,1.0768629717795